In [ ]:
from scipy.sparse import csr_matrix, lil_matrix, load_npz, save_npz, hstack, coo_matrix
import numpy as np 
import pandas as pd 
from tqdm import tqdm 
import os

In [ ]:
tuner_output_baseline_w_aux = '../04_aux_data_preperation/baseline_plus_aux_data/'

path = os.path.join(tuner_output_baseline_w_aux, 'matrices/cls/cls_T10_y.npz')
y_aux = load_npz(path)
y_aux

In [ ]:
y_sparse_main_fold0 = lil_matrix(y_aux.shape)

In [ ]:
# Generate the y-sparse matrix for inference 
# Main tasks + fold 0 to limit the amount of predictions

In [ ]:
path = os.path.join(tuner_output_baseline_w_aux, 'matrices/cls/cls_T11_fold_vector.npy')
folds = np.load(path)

In [ ]:
path = './files/mapping/baseline_image_model_baselineaux_task_mapping.csv'
df_matching = pd.read_csv(path)

In [ ]:
df_matching[['cont_classification_task_id', 'cont_classification_task_id_image', 'cont_classification_task_id_baseline', 'baseline_compliant_input_assay_id_image']].head()

In [ ]:
# main tasks 
# mind the difference between these two !
# cont_classification_task_id is the baseline + aux model 
display(df_matching.query('validity_0 >= 0').query('validity_1 >= 0')['cont_classification_task_id'].tail())
display(df_matching.query('validity_0 >= 0').query('validity_1 >= 0')['cont_classification_task_id_baseline'].tail())

In [ ]:
lst = [0
       ,0.2
       ,0.4
       ,0.5
       ,0.6
       ,0.7
       ,0.8
       ,0.9
       ,0.95
       ,0.99
      ]

In [ ]:
for i,l1 in tqdm(enumerate(lst)):
    l2 = l1
    print(df_matching.query('validity_0 > @l1').query('validity_1 > @l2')['cont_classification_task_id_baseline'].nunique())

In [ ]:
base_path = '../04_aux_data_preperation/baseline_plus_aux_data/matrices/cls/fold_0'
os.makedirs(base_path, exist_ok=True)


In [ ]:
# new approach : for inference, only consider the main tasks that correspond to aux tasks meeting the criteria 
# instead of one general sparse y matrix 


for i,l1 in tqdm(enumerate(lst)):
    y_sparse_main_fold0 = lil_matrix(y_aux.shape)
    l2 = l1
    #for j,l2 in enumerate(lst): 
    file_core = 'ppv{}_npv{}'.format(l1,l2).replace('.','_')
    col_idxs = set(df_matching.query('validity_0 > @l1').query('validity_1 > @l2')['cont_classification_task_id_baseline'].dropna())
    for col in col_idxs :
        y_sparse_main_fold0[folds == 0,col] = 1
    path = os.path.join(base_path, 'y_sparse_main_tasks_fold0_{}'.format(file_core))
    save_npz(path,csr_matrix(y_sparse_main_fold0))

In [ ]:
y_sparse_main_fold0 = lil_matrix(y_aux.shape)

col_idxs = set(df_matching['cont_classification_task_id_baseline'])
for col in col_idxs :
    y_sparse_main_fold0[folds == 0,col] = 1
path = os.path.join(base_path, 'y_sparse_main_tasks_fold0_baseline.npz')
save_npz(path,csr_matrix(y_sparse_main_fold0))